### Basic example on how to use OMERO Python language bindings to interact with OMERO.tables
See https://docs.openmicroscopy.org/latest/omero/developers/Python.html

### Import Packages required to connect to OMERO

In [1]:
import omero
from omero.gateway import BlitzGateway
from ipywidgets import widgets
from getpass import getpass

### Create a connection to an OMERO server

In [2]:
def connect(hostname, username, password):
    """
    Connect to an OMERO server
    :param hostname: Host name
    :param username: User
    :param password: Password
    :return: Connected BlitzGateway
    """
    conn = BlitzGateway(username, password,
                        host=hostname, secure=True)
    conn.connect()
    conn.c.enableKeepAlive(60)
    conn.SERVICE_OPTS.setOmeroGroup("-1")
    return conn

HOST = 'wss://workshop.openmicroscopy.org/omero-ws'
conn = connect(HOST, input("Username: "),
               getpass("OMERO Password: "))
print("Connected as {}".format(conn.getUser().getName()))

Username: trainer-1
OMERO Password: ········
Connected as trainer-1


## Load the file annotations linked to the specified data
OMERO.tables are file attachments.

In [3]:
data = conn.getObject("Plate", input("Enter ID:"))

Enter ID:55


In [4]:
attachments = {}
for ann in data.listAnnotations():
    if ann.OMERO_TYPE == omero.model.FileAnnotationI and ann.getNs() == "openmicroscopy.org/omero/bulk_annotations":
        attachments[ann.getFile().getName()] = ann.getFile().getId()
print(attachments)

{'idr0002_cellprofiler': 98319}


## Select the table to load

In [5]:
def dropdown_widget(values,
                    dropdown_widget_name,
                    displaywidget=False):

    alg_sel = widgets.Dropdown(
        options=values,
        value=values[0],
        description=dropdown_widget_name,
        disabled=False,
    )
    if displaywidget is True:
        display(alg_sel)
    return alg_sel


selection = dropdown_widget(
    list(attachments.keys()),
    'Select table: ', True
)

Dropdown(description='Select table: ', options=('idr0002_cellprofiler',), value='idr0002_cellprofiler')

In [6]:
selected_id = attachments[selection.value]
print(selected_id)

98319


## Load the selected table 

In [7]:
def load_table(conn, fileid):
    """
    Query a table specified by fileid
    Returns a dictionary with query result if successful, error information
    otherwise.
    :param conn: The BlitzGateway
    :param file_id: The table's id
    :return: The table
    """
    r = conn.getSharedResources()
    ctx = conn.createServiceOptsDict()
    ctx.setOmeroGroup("-1")
    return r.openTable(omero.model.OriginalFileI(fileid), ctx)

table = load_table(conn, selected_id)

## Retrieve  rows according to a specific query

In [8]:
rowCount = table.getNumberOfRows()
print("Row count:", rowCount)
name = "Cell_Count"
# Prepare the query
query = "(%s > 5) & (%s <= 20)" % (name, name)
query_rows = table.getWhereList(query, variables={}, start=0, stop=rowCount, step=0)
data = table.readCoordinates(query_rows)
for col in data.columns:
    if name == col.name:
        print("Query Results for Column: ", col.name)
        for v in col.values:
            print("   ", v)

Row count: 96
Query Results for Column:  Cell_Count
    14.0
    20.0
    16.0
    11.0
    15.0
    17.0
    17.0
    20.0
    18.0
    16.0


## Close the table 

In [9]:
table.close()

## Close the connection to the OMERO server

In [10]:
conn.close()

### License
Copyright (C) 2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.